One to thing to examine is if a mare follows a good producer is she likely to follow as well? if that is the case i will need to change my scrpaer and include the dams name somehow in the niece/nephew thier to see is the mare on offer following good or bad producers.

Also from the first model I did, it was clear that BT is overhyped in mare prices, is that hype filtering down to thier prog?


**can get a nice bit of the prog tiers dams by merging in sale data from the sale sheets. thank god for that.** but still could look into scraping the extra pages that where sorted on stack over flow in january.

# Intro

The bulk file was taking to long to load so I will just focus on the mare 2019 sale. I will try look back and predict mare ability. both before they have any runners as well when they have a few etc. It must be noted that the generation/tier analised here will be confusing when looking back in time. A large part of this will be looking at the dams of the mares on offer (there best mark, there dams mean prog rating (2nd dams of mare on offer)). There was more complete data on this thier.

In [1]:
##importing packages needed for this project
from bs4 import BeautifulSoup as bs
import requests
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import PredictionErrorDisplay
from sklearn.model_selection import KFold, cross_val_score, RepeatedKFold
from sklearn.datasets import make_regression
from sklearn.ensemble import AdaBoostRegressor
import sklearn.metrics as metrics

warnings.filterwarnings('ignore')
sns.set_theme(style="whitegrid")
sns.color_palette("Spectral", as_cmap=True)
sns.set()

In [ ]:
##below reads in all the sheets into a dictionary, with sheet names as the keys
all_data_sets = pd.read_excel('/content/drive/MyDrive/horse_racing/sale_work/bulk_scrape/2018_raw.xlsx', sheet_name=None)
print(all_data_sets.keys())

#EDA - Broodmare ability
starting by getting some metrics that can help predict a mares broomare ability.
mean prog rating to date, dam mean prog rating to date, race abilty, dam ability. are some areas to start. prog sales price another area to look at as well.

dam sire data would be too outdated at this stage

### Mean max rating correlation X foal number.
Will cumilitavly look at a mares mean prog max rating, I will then compare that to their end rating and see when do we start getting correlation. Starting by looking at the "own_sib" thier as that will have more suitable data.

In [ ]:
##the data below will have been updated and contains info after the mares sales. will have to use the full form data set and filter down the data we would have had at the time later.
own_sib_basic_form_df = all_data_sets['own_sib_form_basic']
own_sib_basic_form_df = own_sib_basic_form_df.loc[own_sib_basic_form_df['mainType'] == 'FLAT']
own_sib_basic_form_df = own_sib_basic_form_df.sort_values(['sale_lot_no', 'horseYob'])
own_sib_basic_form_df.head(5) #can cases like horses like Across (USA) who won a us race and got plenty of money but a 0 rpr. I will try remove those examples. prob important to keep the shit 0's in there tough.
#there is just over 200 counts of 0 bestRpPostmark i will remove the ones who got more then 4 grand prize money as they likely should not be in there.

In [ ]:
#remove 0 and ones who got prize money
own_sib_basic_form_df_2 = own_sib_basic_form_df.loc[~((own_sib_basic_form_df['totalPrizeMoney'] > 4000) & (own_sib_basic_form_df['bestRpPostmark'] == 0))]
own_sib_basic_form_df_2.head(8);

In [ ]:
running_mean_prog_rating = own_sib_basic_form_df_2.dropna(subset = 'bestRpPostmark').groupby(['sale_lot_no'])['bestRpPostmark'].expanding().agg({'mean', 'count'})
running_mean_prog_rating.head()

In [ ]:
running_mean_prog_rating = running_mean_prog_rating.reset_index().drop(columns = 'level_1')
running_mean_prog_rating.head()

In [ ]:
running_mean_prog_rating_foal_number = running_mean_prog_rating.pivot(columns ='count', values = 'mean', index = 'sale_lot_no')
running_mean_prog_rating_foal_number.head(10)

In [ ]:
dams_mean_prog_max_rating = own_sib_basic_form_df_2.dropna(subset = 'bestRpPostmark').groupby(['sale_lot_no'])['bestRpPostmark'].agg({'count','mean'}).rename(columns = {'count':'count_dams_prog_ratings', 'mean':'mean_dams_prog_rating'})
dams_mean_prog_max_rating.head()

In [ ]:
dams_running_mean_prog_rating_foal_number = running_mean_prog_rating_foal_number.join(dams_mean_prog_max_rating)
dams_running_mean_prog_rating_foal_number.head()

In [ ]:
dams_running_mean_prog_rating_foal_number.corr()

The above table suggests it is dangerous to judge a mare after 1 foal. even 2/3 is risky. After 4/5 foals we have a fair idea. Baring in mind horse age is a factor in sales, id be happy to chance a mare after 3 particularly if there were nice sires to come or if the coming progeny sold well. after 4 racing foals we have a strong idea, baring in mind actually the above table would have the max ratings after the horses retired and if we are looking at a fertile mare with 3 runners then thered only haven been one to get the chance to race at 4 so probably shouldnt be too harah on her (element tough of things have to happen quick in this game which balances things out a touch).

**The above suggests our model might be better not including mares with only 1 runner**. wont get too bogged down on it anyway.

### foal number vs max rating
quick look at when does a mare produce their best foals. will compare the foal number to average of what the mare achieved and then take the average of the columns. if i didnt compare it mare for mare i suspect the older columns would be skewed by the market keeping onto mares.


In [ ]:
ratings = own_sib_basic_form_df_2.dropna(subset = 'bestRpPostmark')[['sale_lot_no', 'bestRpPostmark', 'horseYob', 'horseName']]
ratings.head(20)

In [ ]:
ratings['foal_count'] = own_sib_basic_form_df_2.dropna(subset = 'bestRpPostmark').groupby(['sale_lot_no'])['bestRpPostmark'].cumcount() + 1
ratings.head()

In [ ]:
dams_foal_number_ratings = ratings.pivot(index = 'sale_lot_no',  columns = 'foal_count', values = 'bestRpPostmark')
dams_foal_number_ratings = dams_foal_number_ratings.join(dams_mean_prog_max_rating.drop(columns = 'count_dams_prog_ratings'))
dams_foal_number_ratings.head()

In [ ]:
dams_foal_number_ratios = dams_foal_number_ratings.div(dams_foal_number_ratings['mean_dams_prog_rating'], axis = 0)
dams_foal_number_ratios.head(10)

In [ ]:
dams_foal_number_ratios.mean(axis = 0).plot(marker = 'o')

#### Foal number ratings
Very interesting how the trend goes down with time. I suspect the variance at the end will be down to small samples. The above slope is prob worsened by the fact that a mare still producing is prioven and prob going to better sire but yet the trend is still slipping.

The above would also dismiss the first foal theory in the industry. First foals do very well, 2nd best overall. **Must see is that played out in the market at a later stage.** At this moment in time its not the biggest edge to my eye when you look at the scale, **surely youd be better off seing the mares first 5 foals and knowing shes good** rather then guessing what she can do.

**However Tom Wilson reasearch wouls suggest the above is more significant then I initially tought** .As it stands he thinks theres an edge in the market striking early, howevrer given how hard ive found it to predict broodmare ability below I have to think the solution lies in the middle somewhere, I realy suspect I need to see a mares progeny ratings/sales to be confident buying a mare at present.
https://tomwilson1986.medium.com/unexposed-pedigrees-the-explainability-paradox-0253e7fd9449

### %above 80,90,100 etc

In [ ]:
above_80 = pd.DataFrame()
for i in dams_foal_number_ratings.columns[:-1]:
  dams_foal_number_ratings.loc[dams_foal_number_ratings[i] >= 80, f"{i}_above_80"] = 1
  dams_foal_number_ratings.loc[dams_foal_number_ratings[i] < 80, f"{i}_above_80"] = 0
  above_80[f"{i}_above_80"] = dams_foal_number_ratings[f"{i}_above_80"]
  dams_foal_number_ratings = dams_foal_number_ratings.drop(columns = f"{i}_above_80")

above_80_per = above_80.expanding(axis=1).mean() # abit anoying we loose the nans. will examine the table now anyway for corelation
above_80_per.head()

In [ ]:
##column 16 represents the final % vale above 80 for all mares above. so it will do as the one to compare correlation against.
above_80_per.corr()

simlar story as when looking at the ratings. **but % above 80 looks more correlated** . will have to come up with a way to split the df up so that we use the first x foals to predict the next y foals. atm what I have been looking at is the first x being used to predict the overall score which has the x amount in it making it more accurate. that has been the case for both mean ratings and above 80 rating.

### predicting future using past
as stated above will try and use past data that predicyts future data w/o the past data getting caught up in it.

In [ ]:
##will reduce it mares with x or more foals for now. hopefully not mudering sample size here.
foals = 10
busy_mares = dams_foal_number_ratings.dropna(subset = [foals])
correlation = []
print(f"sample size : {busy_mares.shape[0]}")
for i in range(1,foals+1):
  first_x_foals = busy_mares.iloc[:,:i]
  first_x_foals_mean = first_x_foals.mean(axis=1)
  last_y_foals = busy_mares.iloc[:,i:-1]
  last_y_foals_mean = last_y_foals.mean(axis =1)
  before_after = pd.concat([first_x_foals_mean, last_y_foals_mean], axis = 1)
  #print(before_after.head())
  #print(before_after.corr())
  correlation.append(before_after.corr().iloc[0,1])
sns.lineplot(correlation, marker = 'o')
#must note the below graph to add 1 to the x axis to represent how many past foal ratings used to predict future ratings.

we can see that the correlation has gone down dramtically when you take the old data out of the overall mean. thankfully i looked back on this. we can see as well the more foals the better. in order to keep sample size up i had it cut off at 7 foals. since the tail dipped below i am going to look at mares with 10 foals to see was 6 foals the point where it tailed off. **still a correlation of 0.3 ish for 4 foals** is decent (remember add 1 to above graph for number of past foals in sample)

**It has platowed at 6 foals. im sure the more the better in realty but those oppurtunities are rare.**

### mares racing ability versus her broodmare average prog.

merging in dams ratings as we have been looking at the ability of the dams of the mares on offers broodmare ability.

In [ ]:
sale_df = all_data_sets['sale_df']
dams_aunt_uncles_form_basic_df = all_data_sets['dams_aunt_uncles_form_basic']
dams_aunt_uncles_form_basic_df = dams_aunt_uncles_form_basic_df.loc[dams_aunt_uncles_form_basic_df['mainType'] == 'FLAT']
dams_aunt_uncles_form_basic_df_2 = dams_aunt_uncles_form_basic_df.loc[~((dams_aunt_uncles_form_basic_df['totalPrizeMoney'] > 4000) & (dams_aunt_uncles_form_basic_df['bestRpPostmark'] == 0))]
##above cleans out the cases that have a rating of 0 when they deserved a rating (might have got one due the area they raced in)
dams_names = sale_df['dam_style_name']
dams_form_basic_df = dams_aunt_uncles_form_basic_df_2.loc[dams_aunt_uncles_form_basic_df_2['horseName'].isin(dams_names)]

dams_ratings = dams_form_basic_df[['horseName', 'sale_lot_no', 'bestRpPostmark']]
dams_ratings.head()

In [ ]:
sns.histplot(dams_aunt_uncles_form_basic_df, x = 'bestRpPostmark')
sns.histplot(dams_aunt_uncles_form_basic_df_2, x = 'bestRpPostmark', kde = True) ##just a quick demo of how we removed a few 0's without removing them all, as hoped.

In [ ]:
sns.histplot(dams_ratings, x = 'bestRpPostmark', kde = True)

 The kde line shows here how the industrty doesnt really breed off poor mares. the portion of ratings below 40 is much lower in the dams dataset compared to the dam_aunts_uncles part. which suggests fillies that where useless on the track didnt go the shed as much as others. makes sense but industry looks to overplay this based on Tom wilsons analysis.

In [ ]:
dams_df = dams_ratings.merge(dams_mean_prog_max_rating, on = 'sale_lot_no', how = 'left')#.dropna()
print(f"sample size : {dams_df.dropna().shape[0]}")
dams_df.head()

In [ ]:
dams_df.drop(columns = {'horseName', 'bestRpPostmark_bin', 'saleDate'}).corr()

0.15 correlation for a large sample. shows that proven mares are better then faster mares. will see is there multicolinearity. im sure there is, will see do they cancel each other out to some degree. i.e is an unproven fast mare going to be twice (going for twice as Im kind of taking prog ratings as correlation 0.3) as likely a success then a slow mare.

**hang on am i getting cought up here in correlation rather then slope again?**
will stay going with other things for now. how else can i predict a mares abilty?? dams broodmare ability.

#predicting dams broodmare ability
closer look at the rating of the mare in question and the mean of thier dams progeny ratings.




In [ ]:
scnd_dams_prog_ratings = dams_aunt_uncles_form_basic_df.groupby('sale_lot_no')['bestRpPostmark'].agg({'count', 'mean'}).rename(columns = {'count': 'count_scnd_dams_prog_ratings', 'mean': 'scnd_dams_mean_prog_ratings'})
scnd_dams_prog_ratings.head()

In [ ]:
dams_df.head()

In [ ]:
#merge into a df. start building a df that will be used to model a mares future foal ratings. this is starting to take shape. i will be confused in the morning.
dams_df = dams_df.set_index('sale_lot_no')
dams_df = dams_df.join(scnd_dams_prog_ratings)
dams_df.head(7)

In [ ]:
scnd_dams_prog_ratings.head()

In [ ]:
dams_df.head()

In [ ]:
dams_df.index.value_counts().sort_values()

In [ ]:
dams_df[['mean_dams_prog_rating', 'scnd_dams_mean_prog_ratings','bestRpPostmark']].corr()

The above suggests a mares dams producing ability is better correlated then the mares itself racecourse ability. The slopes will be interesting. will build a small model to have a look now.

In [ ]:
dams_df.drop(columns = {'count_dams_prog_ratings', 'count_scnd_dams_prog_ratings'}).hist()

In [ ]:
data = dams_df.dropna()
x = data[['bestRpPostmark', 'scnd_dams_mean_prog_ratings']]
y = data['mean_dams_prog_rating']
print(data.shape)

In [ ]:
coefficients = pd.DataFrame()
model = LinearRegression()
model.fit(x, y)
r2_score = model.score(x, y)
print(f"r2_score : {r2_score}")
coef = pd.DataFrame(model.coef_, index = [x.columns], columns = ['coeficient']).transpose()
coef

In [ ]:
dams_df['bestRpPostmark_bin'] = pd.cut(dams_df['bestRpPostmark'], bins = [0, 45,55,65,75,85,95,105,135])
dams_df.groupby(['bestRpPostmark_bin'])['mean_dams_prog_rating'].mean().plot(marker = 'o') #5 lb swing for such a wide range of ratings. very low slope in the context of the cost of mares.
##gas we have the same finding as TW the useless ones still produce better then the poor ones. I have to guess this is beacuse the mares where tried based on good homework but a non hiertery issue held them back. amazing, you can spend millions on a superstare mare but only be 3lb better then a injured one.

##### Note
The above sugeests that the dam of the mares ability to produce good horses is more impoirtant then the mare itselfs racecourse ability. The slope is 3 times that of the mares racing ability and the correlation is higher too boot (0.2 vs 0.15). **This is a cool finding and one that should be looked into to see how the market weighs these things.** Also must be noted how shite the model is at predicting a mares abilty using the above metrics. **This is dissapointing.**
but its something the below found as well. **sire data is likely to be a help, i.e mares going to better sires will help them**. note: Not the quality of racehorse a broodmare was, that is something i have over looked at when predicting sires. **I was using mares ratings, in hindsight a mares broodmare ability would have been a much better uplift metric. Thats something I will try note for 2025.**

https://tomwilson1986.medium.com/does-the-ability-of-the-dam-correlate-with-the-ability-of-her-offspring-d1964dfa1e39

**Looking at my 2023 analysis it looks like the market is picking up the prog ratings is more significant**. but I should have a better look, the decimals where tiny. Ill prob build a model using just the 2 metrics. Shame that the data above consists of the mares dams and 2nd dams rather the mares on offer as then i could have built a model here straighaway.

Also I should note I can only imagine that the number of foals is sigificant for the reliabilty of the dams_mean_prog_rating as a predictor variable. just like the mares on offer as well.

Another angle. it effects racehorses. does it effect what foal number a mare is as a broodmare?

In [ ]:
##want to view the above on a scatterplot.
sns.lmplot(data = data , x = 'bestRpPostmark', y = 'mean_dams_prog_rating')
sns.lmplot(data = data , x = 'scnd_dams_mean_prog_ratings', y = 'mean_dams_prog_rating')

In [ ]:
##quick check for multicolinearity. #of course these are correlated as the bestRpPostmark figure is one of the numbers that calculates the dams_mean_prog_rating, they would be less corelated the larger the dams foals sample size
x.corr()

In [ ]:
##will build a model using 1 variable each.
coefficients = pd.DataFrame()
model = LinearRegression()
model.fit(x.drop(columns = {'scnd_dams_mean_prog_ratings'}), y)
r2_score = model.score(x.drop(columns = {'scnd_dams_mean_prog_ratings'}), y)
print(f"r2_score : {r2_score}")
coef = pd.DataFrame(model.coef_, index = [x.drop(columns = {'scnd_dams_mean_prog_ratings'}).columns], columns = ['coeficient']).transpose()
coef

In [ ]:
##will build a model using 1 variable each.
coefficients = pd.DataFrame()
model = LinearRegression()
model.fit(x.drop(columns = {'bestRpPostmark'}), y)
r2_score = model.score(x.drop(columns = {'bestRpPostmark'}), y)
print(f"r2_score : {r2_score}")
coef = pd.DataFrame(model.coef_, index = [x.drop(columns = {'bestRpPostmark'}).columns], columns = ['coeficient']).transpose()
coef

we can see that the r scores with them individually are lower. however there is much of a muchness really when looking at the dams mean prog ratings. and its usually best to have a lesser amount of variables. the coefficient is higher when you dont include rating as well. contemplated stripping out the mares rating from the dams mean part to reduce multicolinearity of the variables but whats the point. the rating makes a fraction of the impact that the dams mean prog rating does so why negate that variable just to complecate the model and more then likely just have a lesser model.

In [ ]:
dams_df.head()

In [ ]:
## will filter on both counts and see how that effects correlation
for i in range(1,11):
  temp_df = dams_df.loc[(dams_df['count_dams_prog_ratings'] > i) & (dams_df['count_scnd_dams_prog_ratings'] > i)]
  print(f"count: {i}", temp_df[['mean_dams_prog_rating', 'scnd_dams_mean_prog_ratings']].corr())

### Yearling prices to help predict racehorse abilty - reduce nans
if we can use yearling price as a predictor we can use it to fill in missing data (something might nvr of made track and wew could estimate what it would have achieved) Or it could be used to estimate what it will do going forward.

In [ ]:
dams_aunt_uncles_sales = all_data_sets['dams_aunt_uncles_sales']
dam_sales = dams_aunt_uncles_sales.loc[dams_aunt_uncles_sales['horseName'].isin(dams_names)] ## have the dams names from earlier
dam_yr_sales = dam_sales.loc[dam_sales['horseAge'] == 1]
print(f"df size: {dam_yr_sales.shape}")
dam_yr_sales.head()

In [ ]:
dam_yr_sales.columns

In [ ]:
dam_yr_sales = dam_yr_sales[['saleDate', 'priceGbp', 'sale_lot_no', 'horseYob']].set_index('sale_lot_no')
dam_yr_sales_sold = dam_yr_sales.loc[dam_yr_sales['priceGbp'] > 200]
dam_yr_sales_sold.head() ###make sure this is rigth, similar code was well off below.

In [ ]:
dams_df = dams_df.join(dam_yr_sales_sold)
dams_df.head()

In [ ]:
print(f"sample size: {dams_df[['priceGbp','bestRpPostmark']].dropna().shape[0]}")
dams_df[['priceGbp','bestRpPostmark']].corr()

In [ ]:
dams_df['log_dams_yr_price'] = np.log(dams_df['priceGbp'])
print(f"sample size: {dams_df[['log_dams_yr_price','bestRpPostmark']].dropna().shape[0]}")
dams_df[['log_dams_yr_price', 'bestRpPostmark']].corr()

In [ ]:
sns.lmplot(data = dams_df.loc[dams_df['priceGbp'] > 0], x = 'log_dams_yr_price', y ='bestRpPostmark') #removing when price is 0 as log 0 is inf.
###can get a better sample size to check this out. Ive murdered sample size here.

In [ ]:
dams_df.loc[dams_df['priceGbp'] > 0].groupby('bestRpPostmark_bin')['log_dams_yr_price'].mean().plot(marker = 'o') #removing when price is 0 as log 0 is inf.

In [ ]:
dams_df.loc[dams_df['priceGbp'] > 0].groupby('bestRpPostmark_bin')['log_dams_yr_price'].count() #removing when price is 0 as log 0 is inf.

Im very suprised at how poor yearling prices are correlted to predicting bestRP mark. can this really be believed?

anyway the main take away from the above is that its going to be too hard to use sale price to enhance missing data. **yr sale price im sure has a place in predicting a mares profitability.** And will be used again below.

I will have a better look at yearling/young horse price below.

In [ ]:
##dont have to worry about dams as if a horse isnt named then it womt have a rating, dam names was used to create a ne wid column in previous researh.
sales = dams_aunt_uncles_sales[['horseName', 'horseYob', 'priceGbp', 'horseAge']]
young_sales_dams_aunts_uncles = sales.loc[sales['horseAge'] <= 2]
max_young_sales_dams_aunts_uncles = young_sales_dams_aunts_uncles.groupby('horseName')['priceGbp'].idxmax().dropna()

In [ ]:
young_sales_dams_aunts_uncles.groupby('horseName')['priceGbp'].idxmax().dropna()
###this is fucked up and making a fuck of the whole thing.

In [ ]:
young_sales_dams_aunts_uncles['priceGbp'].max()

In [ ]:
max_young_sales_dams_aunts_uncles.sort_values().head(-15)

In [ ]:
sale_rating_data = max_young_sales_dams_aunts_uncles.reset_index().merge(dams_aunt_uncles_form_basic_df_2[['horseName','bestRpPostmark', 'distanceFurlong', 'stakesWinnerOrPlaced']], on = 'horseName', how = 'inner')
sale_rating_data.head()

In [ ]:
sale_rating_data['priceGbp'].max()

In [ ]:
sns.lmplot(data = sale_rating_data, x = 'priceGbp', y = 'bestRpPostmark')
#####seems to be a massive error here. need to clean data

staggering negetive correlation. this wasnt played out when pricing horses.

##### To do/ideas - predicting broodmare ability



*   first is build a model guesiing a mares future foal ratings. will need a few
*   models based on the stage of her carear.
*   sisters of dams. if the sister is a good dam how strongly correlated is that?
*   have been looking alot at means. how about maxes? i feel the market rewards the best out of a mare. which is wrong from a stats point of view but have to play the game. I just have to try and see where race course ability ties into a broodmare ability/reward in sales ring.
* Tom wilson does alot on winners.
*   **more suitable damsire stats might help**. right now would only have outdated data. have to assign time to scraping the data bettet. dam sire stats from a past point of view. then also the lower tiers/generations being able to merge in mare stats. I didmt even use the stats on offer here. i could prob use them on newer sales that I have scraped.







#EDA mare profitability

knowing what we know about predicting broodmare ability using ratings and thier dams prog ratings (correlated).... its not very accurate unbless having 3/4 ish foals of a sample size, the more the better. Im going to look at what mares sold cheap but thier stock went on to make money and try see why that would have happened, or the edge might just be roll the dice for small money as the market over values too many things.

will be looking at what makes a foal or yearling profitable. mare ability will surely be an overplayed factor.  


In [ ]:
#Im goinh to start a generation lower here. as we have dams name in the sales column. will be easy to shift up/down a generation.
own_sib_sales = all_data_sets['own_sib_sales']
own_sib_sales['id'] = own_sib_sales['damName'].astype(str) + "_" + own_sib_sales['horseYob'].astype(str) #creating an id that individualises the horses sales
own_sib_sales_young = own_sib_sales.loc[own_sib_sales['horseAge'] <= 2]
own_sib_sales_young.head()

In [ ]:
max_own_sib_sales_young = own_sib_sales_young.groupby('id')['priceGbp'].idxmax().dropna()
max_own_sib_sales_young.head()

In [ ]:
dams_mean_prog_max_rating.head()

In [ ]:
dams_df.head()

##### Mare profitablity to do

keep going haha. just leaving space here for now


#### overall to do

new section - look at sale. see what stats the mares had coming in and test and a few theories. then see what the future held for them.